In [200]:
import requests
import pandas as pd
import numpy as np

In [96]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

I put the wikipedia Table into a Pandas Dataframe

In [182]:
df_Toronto=pd.read_html(source)[0]

In [183]:
df_Toronto.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


Now i drop th rows with not assigned Borough, then I change "/" for "," in the Neighborhood column, and finally I assign Borough name to Not Assigned Neighborhoods

In [184]:
for i in range(df_Toronto.shape[0]):
    if df_Toronto['Borough'][i]=='Not assigned':
        df_Toronto.drop([i],inplace=True)
    i=i+1


In [185]:
df_Toronto.reset_index(drop=True,inplace=True)

In [186]:
for i in range(df_Toronto.shape[0]):
    count=0
    nb1=df_Toronto["Neighborhood"][i]
    for char in nb1:
        if char=="/":
            count=count+1
    nb2=nb1.replace("/",",",count)
    df_Toronto=df_Toronto.replace(nb1,nb2)

In [187]:
for i in range(df_Toronto.shape[0]):
    nb1=df_Toronto["Neighborhood"][i]
    if nb1=="Not assigned":
        nb2=df_Toronto["Borough"][i]
        df_Toronto=df_Toronto.replace(nb1,nb2)

In [188]:
df_Toronto #This is how my DataFrame looks like:

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [192]:
df_Toronto["Latitude"]=None
df_Toronto["Longitude"]=None

In [206]:
path = "https://cocl.us/Geospatial_data"
df_latlon=pd.read_csv(path)


In [209]:
for i in range(df_Toronto.shape[0]):
    for j in range(df_latlon.shape[0]):
        if df_latlon["Postal Code"][j]==df_Toronto["Postal code"][i]:
            df_Toronto["Latitude"][i]=df_latlon["Latitude"][j]
            df_Toronto["Longitude"][i]=df_latlon["Longitude"][j]
        

In [210]:
df_Toronto

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6537,-79.5069
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.6627,-79.3216
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.6363,-79.4985


In [ ]:
CLIENT_ID = 'I2YNM3MPROP1XXRKZKCCVIJ4VMYYA4V3FNQNHOLKMST414TP' 
CLIENT_SECRET = 'LT1WQHJXXDZWPOYNO50XBGN0JIEX5QNQ4FQIOI4EWH3OZQ5C'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### I Borrow the GetNearbyValues function from the Lab to get all the venues for each neighborhood.

In [212]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now I call the function with its parameters

In [213]:
nb_names=df_Toronto["Neighborhood"]
nb_lats=df_Toronto["Latitude"]
nb_longs=df_Toronto["Longitude"]

toronto_venues=getNearbyVenues(nb_names,nb_lats,nb_longs)

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
Do

In [234]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Now I analize each neighbourho

In [279]:
toronto_nb_venues = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_nb_venues['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_nb_venues['Latitude'] = toronto_venues['Latitude'] 
toronto_nb_venues['Longitude'] = toronto_venues['Longitude'] 
toronto_nb_venues.columns.get_loc("Neighborhood") #Search "Neighborhood" Column index
fixed_cols = [toronto_nb_venues.columns[161]] + list(toronto_nb_venues.columns[:161])+list(toronto_nb_venues.columns[162:]) #Put Neighborhood column in the first column of the DF
toronto_nb_venues = toronto_nb_venues[fixed_cols]
toronto_nb_venues

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### I group the venues in their Neigbourhoods and calculate the mean frequency of this venues

In [282]:
toronto_groups = toronto_nb_venues.groupby('Neighborhood').mean().reset_index()
toronto_groups.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.25,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

#### We print each Neighbourhood and its top 5 common venues

In [285]:
num_top_venues = 5

for hood in toronto_groups['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_groups[toronto_groups['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0              Breakfast Spot  0.25
1   Latin American Restaurant  0.25
2                Skating Rink  0.25
3                      Lounge  0.25
4  Modern European Restaurant  0.00


----Alderwood , Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11
2            Pharmacy  0.11
3                 Pub  0.11
4         Coffee Shop  0.11


----Bathurst Manor , Wilson Heights , Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2          Diner  0.05
3    Bridal Shop  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.09
2      Sandwich Place  0.09
3  Italia

#### Borrow the function from the lab that gets 10th most common venues for each neighbourhood

In [286]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Using this function I make a new Dataframe showing each neighborhood and its 10th most common venues

In [288]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

nb_venues_sorted = pd.DataFrame(columns=columns)
nb_venues_sorted['Neighborhood'] = toronto_groups['Neighborhood']

for ind in np.arange(toronto_groups.shape[0]):
    nb_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_groups.iloc[ind, :], num_top_venues)

nb_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Skating Rink,Latin American Restaurant,Yoga Studio,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
1,"Alderwood , Long Branch",Pizza Place,Pub,Pharmacy,Gym,Skating Rink,Coffee Shop,Sandwich Place,Athletics & Sports,Yoga Studio,Dessert Shop
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pet Store,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Diner,Restaurant,Ice Cream Shop,Deli / Bodega
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Yoga Studio,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Italian Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant


#### I set the number of clusters and then applying the KMeans function I get the cluster labels for each neighborhood

In [322]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_clustering = toronto_groups.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)
kmeans.labels_[0:95] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 0, 4, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 4, 1, 1, 1, 4,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       4, 1, 1, 1, 1, 1, 1], dtype=int32)

#### I insert the labels into a the DataFrame created before and then I merge this DataFrame with th initial one with the NeighborHoods and their Latitude and Longitude so as to make the map later.

In [326]:
nb_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merge = df_Toronto
#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merge = toronto_merge.join(nb_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how="right")
#toronto_merge['Cluster Labels'] = toronto_merge['Cluster Labels'].astype(int32)
toronto_merge.head() # check the last columns!


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,4,Park,Food & Drink Shop,Yoga Studio,Dessert Shop,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.7259,-79.3156,1,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Event Space,French Restaurant,Distribution Center,Spa,Farmers Market
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7185,-79.4648,1,Accessories Store,Clothing Store,Furniture / Home Store,Boutique,Coffee Shop,Athletics & Sports,Women's Store,Arts & Crafts Store,Event Space,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895,1,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Hobby Shop,Burrito Place,Juice Bar,Sandwich Place,Spa,Italian Restaurant


#### I get Toronto geolocation and then create the Map

In [294]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [328]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters